This code uses SVD++ algorithm to recommend game to the player. The rating is obtained by analyzing the sentiment from the reviews given on each games.

In [38]:
import pandas as pd

In [1]:
import requests
# Get all the games that the player played
def getownedgames(idnum):
    """Takes the id number of a steam user and returns the id num of games they own"""
    url='http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key=3A629E52AE594CB777253431189F2967&steamid={}&format=json'.format(idnum)
    r=requests.get(url)
    json_data=r.json()
    return json_data

Get all the application id that is on STEAM website.

In [2]:
api_key='3A629E52AE594CB777253431189F2967' # API key for getting information from Steam
allgame='http://api.steampowered.com/ISteamApps/GetAppList/v0002/?key={}&format=json'.format(api_key)
all_r=requests.get(allgame)
allgame_id=all_r.json()

In [3]:
steamapp_dict=dict()
for num in range(len(allgame_id['applist']['apps'])):
    key=allgame_id['applist']['apps'][num]['appid']
    value=allgame_id['applist']['apps'][num]['name']
    if key in steamapp_dict:
        continue
    else:
        steamapp_dict[key]=value

get data from pickled file

In [4]:
import pickle
file_name='steam_review_bulk.p'
loaded_df=pickle.load(open(file_name,"rb"))

In [5]:
loaded_df.head()

,author,comment_count,language,received_for_free,recommendationid,review,steam_purchase,timestamp_created,timestamp_updated,voted_up,votes_funny,votes_up,weighted_vote_score,written_during_early_access,game_name,developer_response,timestamp_dev_responded
0,76561198418549470,2,english,False,48738870,"Well, I had never played a time-management gam...",True,1549214430,1549236308,True,0,9,0.589138209819793701,False,Elven Legend,NaN,NaN
1,76561198044857549,0,english,False,30095674,This is the third time I have had to leave a r...,True,1487755088,1487755088,False,0,28,0.807962000370025635,False,Bequest,NaN,NaN
2,76561198030980450,0,english,False,49686292,has potiential. needs more cultures as well. h...,True,1553224936,1553224936,True,0,1,0.444444417953491211,False,Empire Architect,NaN,NaN
3,76561198129998583,0,english,False,45343379,Trash.,True,1539976440,1539976440,False,0,1,0.482841163873672485,False,Jelly in the sky,NaN,NaN
4,76561198135538727,0,english,False,48179714,"Not made for 1080p or higher, since text and i...",True,1546910395,1546910395,False,0,1,0.533527672290802002,False,The Last Cargo,NaN,NaN


Because we do not have rating provided in the raw data, we will get the sentiment values from the reviews and assign rating based on that

In [6]:
loaded_df=loaded_df.drop_duplicates()

In [7]:
from textblob import TextBlob

In [8]:
review_sentiment=list()
review_recomd=list()
i=0
for review in loaded_df.review:
    sent = TextBlob(review)
    rating=round(2*(sent.sentiment.polarity+1))
    if sent.sentiment.polarity>0:
        recomd=True
    else:
        recomd=False
    review_sentiment.append(rating)
    review_recomd.append(recomd)

In [9]:
loaded_df['review_sentiment']=review_sentiment
loaded_df['recommend']=review_recomd

In [11]:
from surprise import *

In [10]:
# Surprise takes user, item, and rating as input
df=loaded_df[['author','game_name','review_sentiment']]

In [28]:
# Only take games that have more than 5 reviews into consideration 
min_games_review = 5
filter_games = df['game_name'].value_counts() > min_games_review
filter_games = filter_games[filter_games].index.tolist()

# Only take users that left more than 5 reviews into consideration 
min_user_review = 5
filter_users = df['author'].value_counts() > min_user_review
filter_users = filter_users[filter_users].index.tolist()

df_new = df[(df['game_name'].isin(filter_games)) & (df['author'].isin(filter_users))]
print('The original data frame shape:\t{}'.format(df.shape))
print('The new data frame shape:\t{}'.format(df_new.shape))

The original data frame shape:	(98044, 3)
The new data frame shape:	(8852, 3)


In [14]:
# Make the reader and data form
reader = Reader(rating_scale=(0, 4))
data = Dataset.load_from_df(df_new, reader)

In [35]:
from surprise.model_selection import cross_validate
svdmodel=SVDpp()
cross_validate(svdmodel,data,measures=['RMSE'],cv=3,verbose=False)

{'test_rmse': array([0.48067647, 0.47552341, 0.45936497]),
 'fit_time': (1.4934563636779785, 1.4026877880096436, 1.4840326309204102),
 'test_time': (0.07191944122314453, 0.06203174591064453, 0.05703902244567871)}

In [36]:
from surprise.model_selection import train_test_split
trainset,testset=train_test_split(data,test_size=0.3)

In [37]:
from surprise import accuracy
model=svdmodel.fit(trainset)
predict1=model.test(testset)
accuracy.rmse(predict1)

RMSE: 0.4731


0.47306970619069905

In [19]:
predict_df=pd.DataFrame(predict1)
predict_df['err'] = abs(predict_df.est - predict_df.r_ui)
best_predictions = predict_df.sort_values(by='err')[:10]
worst_predictions = predict_df.sort_values(by='err')[-10:]

In [20]:
best_predictions

,uid,iid,r_ui,est,details,err
1740,76561198035285721,Juice Fresh,2.0,1.999996,{'was_impossible': False},0.000004
292,76561198035285721,RKN Block Me: Telegram,2.0,1.999996,{'was_impossible': False},0.000004
859,76561198301417309,Shadows in the Darkness,2.0,1.999970,{'was_impossible': False},0.000030
1591,76561198013274120,Vernon's Legacy,2.0,1.999916,{'was_impossible': False},0.000084
285,76561198091533948,Fallout 4,2.0,1.999883,{'was_impossible': False},0.000117
1710,76561198069159152,Disgraced,2.0,2.000392,{'was_impossible': False},0.000392
1696,76561198019816374,Rainbow Step,2.0,1.999580,{'was_impossible': False},0.000420
1963,76561198009614137,★ Fallalypse ★,2.0,1.999391,{'was_impossible': False},0.000609
1377,76561198044229474,GabeN: The Final Decision,2.0,2.000771,{'was_impossible': False},0.000771
2048,76561198003033413,Hell`S Little Story,2.0,1.999188,{'was_impossible': False},0.000812


In [21]:
worst_predictions

,uid,iid,r_ui,est,details,err
2249,76561198390832861,Factorio,4.0,2.051539,{'was_impossible': False},1.948461
2607,76561198068849291,The chronicles of Emerland. Solitaire.,4.0,2.047632,{'was_impossible': False},1.952368
2223,76561198091533948,Fallout 4,4.0,1.999883,{'was_impossible': False},2.000117
202,76561198015030636,Fallout 4,0.0,2.045722,{'was_impossible': False},2.045722
1923,76561198011965365,Lilly and Sasha: Curse of the Immortals,4.0,1.950597,{'was_impossible': False},2.049403
1129,76561198157193042,Shift,4.0,1.939000,{'was_impossible': False},2.061000
574,76561198046979223,Pixel Puzzles 2: Birds,4.0,1.932399,{'was_impossible': False},2.067601
225,76561198025301599,Robot Roller-Derby Disco Dodgeball Soundtrack,4.0,1.885280,{'was_impossible': False},2.114720
694,76561198015785536,Black Desert Online SA,4.0,1.711927,{'was_impossible': False},2.288073
281,76561198272538113,Lumber Island - That Special Place,0.0,2.293233,{'was_impossible': False},2.293233


In [24]:
test_id='76561198141485825'
pred=model.predict(uid=test_id,iid="Fallout 4")
score=pred.est

2.067729734134687
user: 76561198141485825 item: Fallout 4  r_ui = None   est = 2.07   {'was_impossible': False}


In [39]:
tests=[[test_id,iid,5] for iid in predict_df["iid"].unique().tolist()]
predictions=model.test(tests)
pred_df=pd.DataFrame(predictions)

In [40]:
test_df=pd.DataFrame(list(zip(testids,testplaytimes)),columns=['ids','playtime']).sort_values('playtime',ascending=False)
playedgame_name=list()
for id_ in test_df['ids']:
    name=steamapp_dict.get(id_)
    playedgame_name.append(name)

In [42]:
removed=[name for name in pred_df['iid'].tolist() if name not in playedgame_name]

In [43]:
pred_df=pred_df.set_index(['iid'])

In [44]:
finaltop10recommendation=pred_df.loc[removed].sort_values(['est'],ascending=False)[:10]

In [45]:
pred_game=finaltop10recommendation.index[0]
pred_rate=finaltop10recommendation['est'][0]
print('Top item for user {0} is {1} with predicted rating {2}'.format(test_id,pred_game, pred_rate))

Top item for user 76561198141485825 is Warhammer 40,000: Sanctus Reach - Sons of Cadia with predicted rating 2.5731488503622937


Improving the model

In [77]:
from surprise.model_selection import GridSearchCV
param_grid={'lr_all' : [0.0001,.001,.01,.1], 'reg_all':[0.025,.05,.1,.5,.8]}
gs=GridSearchCV(SVDpp,param_grid,measures=['rmse','mae'],cv=3)
gs.fit(data)
print(gs.best_params['rmse']['lr_all'],gs.best_params['rmse']['reg_all'])

0.001 0.8


In [78]:
gs.best_params

{'rmse': {'n_factors': 100, 'n_epochs': 120, 'lr_all': 0.001, 'reg_all': 0.8},
 'mae': {'n_factors': 100, 'n_epochs': 120, 'lr_all': 0.001, 'reg_all': 0.8}}

In [82]:
new_model=SVDpp(lr_all=gs.best_params['rmse']['lr_all'],
                reg_all=gs.best_params['rmse']['reg_all'])
cross_validate(new_model,data,verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.4788  0.4877  0.4876  0.4787  0.4644  0.4795  0.0085  
MAE (testset)     0.2920  0.2946  0.2920  0.2827  0.2742  0.2871  0.0076  
Fit time          1.76    1.75    1.88    2.06    1.87    1.87    0.11    
Test time         0.04    0.04    0.06    0.04    0.04    0.04    0.01    


{'test_rmse': array([0.47882086, 0.48772503, 0.48763341, 0.4786779 , 0.46444422]),
 'test_mae': array([0.29202963, 0.29459704, 0.29203005, 0.28272914, 0.27424259]),
 'fit_time': (1.7622883319854736,
  1.7533588409423828,
  1.8817856311798096,
  2.0648488998413086,
  1.8654580116271973),
 'test_time': (0.04020833969116211,
  0.036743879318237305,
  0.05508565902709961,
  0.04463958740234375,
  0.03623700141906738)}

In [49]:
from surprise.model_selection import train_test_split
trainset,testset=train_test_split(data,test_size=0.3)

In [83]:
from surprise import accuracy
model2=new_model.fit(trainset)
predict2=model2.test(testset)
accuracy.rmse(predict2)

RMSE: 0.4769


0.4768878945828707

In [51]:
import pandas as pd
predict_df2=pd.DataFrame(predict2)
predict_df2['err'] = abs(predict_df2.est - predict_df2.r_ui)
best_predictions2 = predict_df2.sort_values(by='err')[:10]
worst_predictions2 = predict_df2.sort_values(by='err')[-10:]

In [57]:
tests=[[test_id,iid,5] for iid in predict_df2["iid"].unique().tolist()]
new_predictions=new_model.test(tests)
pred_df2=pd.DataFrame(new_predictions)

In [59]:
pred_df2=pred_df2.set_index(['iid'])
new_top10recommendation=pred_df2.loc[removed].sort_values(['est'],ascending=False)[:10]

C:\Users\shshi\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  


In [60]:
pred_game2=new_top10recommendation.index[0]
pred_rate2=new_top10recommendation['est'][0]
print('Top item for user {0} is {1} with predicted rating {2}'.format(test_id,pred_game2, pred_rate2))

Top item for user 76561198141485825 is MARS VR(全球使命VR) with predicted rating 2.5100051364363445


In [70]:
top10=pred_df2.sort_values(['est'],ascending=False)[:10]
recommended_games=list()
for name in top10.index.tolist():
    ids=list(steamapp_dict.keys())[list(steamapp_dict.values()).index(name)]
    recommended_games.append((name,"https://store.steampowered.com/app/"+str(ids)))

In [64]:
top10

,uid,r_ui,est,details
iid,,,,
MARS VR(全球使命VR),76561198141485825,4,2.510005,{'was_impossible': False}
Euro Truck Simulator 2 - Danish Paint Jobs Pack,76561198141485825,4,2.505192,{'was_impossible': False}
Mars,76561198141485825,4,2.481897,{'was_impossible': False}
Down Here,76561198141485825,4,2.481482,{'was_impossible': False}
FSX: Steam Edition - Greenland Nuuk Add-On,76561198141485825,4,2.468923,{'was_impossible': False}
Hover 2030,76561198141485825,4,2.434097,{'was_impossible': False}
Hiveswap Friendsim - Volume Eight,76561198141485825,4,2.425076,{'was_impossible': False}
The Grand Canyon VR Experience,76561198141485825,4,2.422966,{'was_impossible': False}
Brilliant Shadows - Part One of the Book of Gray Magic,76561198141485825,4,2.417089,{'was_impossible': False}


In [73]:
print("Based on what other players' are playing a lot, here are some of the games you may like.")
print(pd.DataFrame(recommended_games,columns=["Game Name","Store page"]))

Based on what other players' are playing a lot, here are some of the games you may like.
                                           Game Name  \
0                                    MARS VR(全球使命VR)   
1    Euro Truck Simulator 2 - Danish Paint Jobs Pack   
2                                               Mars   
3                                          Down Here   
4         FSX: Steam Edition - Greenland Nuuk Add-On   
5                                         Hover 2030   
6                  Hiveswap Friendsim - Volume Eight   
7                     The Grand Canyon VR Experience   
8  Brilliant Shadows - Part One of the Book of Gr...   
9                        3 Suicides of Paul Hamilton   

                                  Store page  
0  https://store.steampowered.com/app/596560  
1  https://store.steampowered.com/app/347210  
2  https://store.steampowered.com/app/357400  
3  https://store.steampowered.com/app/387690  
4  https://store.steampowered.com/app/379541  
5  https://s